In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Class 1-samples.zip to Class 1-samples.zip
Saving Class 2-samples.zip to Class 2-samples.zip


In [ ]:
import os

# Rename the files to simpler names
os.rename("Class 1-samples.zip", "class1.zip")
os.rename("Class 2-samples.zip", "class2.zip")

In [ ]:
import zipfile

# Create the main dataset folder
os.makedirs("face_dataset/class1", exist_ok=True)
os.makedirs("face_dataset/class2", exist_ok=True)

# Unzip class1.zip
with zipfile.ZipFile("class1.zip", 'r') as zip_ref:
    zip_ref.extractall("face_dataset/class1")

# Unzip class2.zip
with zipfile.ZipFile("class2.zip", 'r') as zip_ref:
    zip_ref.extractall("face_dataset/class2")


In [ ]:
!pip uninstall numpy -y
!pip install numpy==1.23.5
!pip install opencv-contrib-python==4.5.5.64 --force-reinstall

Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.
db-dtypes 1.4.3 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
geopand

  Using cached opencv_contrib_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 744.3 kB/s eta 0:00:00
Using cached opencv_contrib_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (66.7 MB)
   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/16.9 MB 20.5 MB/s eta 0:00:01ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/16.9 MB 26.6 MB/s eta 0:00:01^C


In [ ]:
import numpy as np
import cv2

print("NumPy version:", np.__version__)
print("OpenCV version:", cv2.__version__)

NumPy version: 1.23.5
OpenCV version: 4.5.5


In [ ]:
import cv2
import numpy as np
import os
from PIL import Image

In [ ]:
def get_images_and_labels(path):
    image_paths = []
    labels = []
    label_dict = {}
    current_label = 0

    for folder_name in os.listdir(path):
        folder_path = os.path.join(path, folder_name)
        if not os.path.isdir(folder_path):
            continue
        label_dict[current_label] = folder_name
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            image_paths.append(img_path)
            labels.append(current_label)
        current_label += 1

    faces = []
    face_labels = []

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    for img_path, label in zip(image_paths, labels):
        try:
            gray_img = Image.open(img_path).convert('L')
            img_np = np.array(gray_img, 'uint8')
            detected_faces = face_cascade.detectMultiScale(img_np)

            for (x, y, w, h) in detected_faces:
                face = img_np[y:y+h, x:x+w]
                faces.append(face)
                face_labels.append(label)
        except Exception as e:
            print(f"Skipping {img_path}: {e}")

    return faces, face_labels, label_dict


In [ ]:
dataset_path = "/content/face_dataset"

faces, labels, label_dict = get_images_and_labels(dataset_path)

print(f"Total faces detected: {len(faces)}")
print("Classes:", label_dict)


Total faces detected: 195
Classes: {0: 'class2', 1: 'class1'}


In [ ]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(labels))

print("Model trained successfully")


Model trained successfully


In [ ]:
def recognize_face(image_path):
    gray = Image.open(image_path).convert('L')
    img_np = np.array(gray, 'uint8')

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(img_np)

    for (x, y, w, h) in faces:
        face = img_np[y:y+h, x:x+w]
        label, confidence = recognizer.predict(face)
        print(f"Prediction: {label_dict[label]} | Confidence: {confidence:.2f}")


In [ ]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded:
    recognize_face(filename)


Saving Screenshot 2025-06-18 003431.png to Screenshot 2025-06-18 003431.png
Prediction: class2 | Confidence: 107.92


In [ ]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded:
    recognize_face(filename)

Saving Aditee Photo.jpeg.jpg to Aditee Photo.jpeg.jpg
Prediction: class1 | Confidence: 201.72
Prediction: class2 | Confidence: 165.66
Prediction: class2 | Confidence: 115.49
Prediction: class1 | Confidence: 148.54
